# **PREDICCIONES AGREGADAS**

In [1]:
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import pickle
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import random as random
import plotly.express as px
import tensorflow as tf
import gzip

def set_output_precision(decimals):
  """
  format the output of the all the data structures
  with an specific number of decimals
  """
  np.set_printoptions(precision=decimals)
  into='{'+':.{}f'.format(decimals)+'}'
  pd.options.display.float_format = into.format
  pass

def plot_ts(df,dfx="Minute",dfy="METS",_title="DF minute x Mets"):
  if not isinstance(df,pd.DataFrame):
    df = pd.DataFrame({'METS': df, 'Minute': range(len(df))})

  plt.figure()
  fig = px.line(df, x = dfx, y = dfy , title = _title)
  fig.update_xaxes(
      rangeslider_visible = True,
      rangeselector = dict(
          buttons = list([
              dict(count=1,label="1y",step="year",stepmode="backward"),
              dict(count=2,label="2y",step="year",stepmode="backward"),
              dict(count=3,label="3y",step="year",stepmode="backward"),
              dict(step="all")
          ])
      )

  )
  fig.show()

def plot_predictions_vs_real(predictions, reals):
  df = pd.DataFrame()
  number_of_points = len(predictions)
  df["time"] = range(0,number_of_points)
  df["participant"] = "prediction"
  df["value"] = predictions
  for i in range(0,number_of_points):
    df.loc[number_of_points+i] = [i,"real",reals[i]]

  plt.figure(1)
  fig = px.line(df, x = "time", y = "value" , title = "predictions vs reals" , color = "participant")
  fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(count=2,label="2y",step="year",stepmode="backward"),
                dict(count=3,label="3y",step="year",stepmode="backward"),
                dict(step="all")
            ])
        )

    )
  fig.show()

In [2]:
READ_LOCAL_DATA = True
COMPUTED_OPTION = 0
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.15
SAVE_RESULTS = False
LOW_DATA = True
SPLIT_INTO_TWO_DAYS = False
MULTI_STEP_FORECAST = False
np.random.seed(42)

In [3]:
if READ_LOCAL_DATA:
  PATH = "Resources/Agregado/"
else:
    #  We start by getting access to the drive
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/TFG/Resources/Agregado/"


if LOW_DATA:
    PATH += "LowData/"

In [4]:
import gzip
documents = ['minuteY','hourY','dayY']
# with open(PATH+"minuteX"+".pkl", 'rb') as file:
#     dataX = np.array(pickle.load(file),np.float32)
file = PATH+"minuteX.pkl.gz"
dataX = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

file = PATH+documents[COMPUTED_OPTION]+".pkl.gz"
dataY = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

In [5]:
if MULTI_STEP_FORECAST:
    PREDICTED_HORIZON = 1
    if COMPUTED_OPTION == 0:
        dataX_nuevo = dataX
        JUMP = 1
    elif COMPUTED_OPTION == 1:
        JUMP = 60
        dataX_nuevo = np.ones(shape=(dataX.shape[0],24))
        for i in range(0,dataX.shape[0]):
            for j in range(0,24):
                dataX_nuevo[i,j] = np.sum(dataX[i,60*j:60*(j+1)])
    else:
        JUMP = 1440
        dataX_nuevo = np.ones(shape=(dataX.shape[0],1))
        for i in range(0,dataX.shape[0]):
            dataX_nuevo[i,:] = np.sum(dataX[i,:])
else:
    JUMP = 1
    if COMPUTED_OPTION == 0:
        PREDICTED_HORIZON = 120
    elif COMPUTED_OPTION == 1:
        PREDICTED_HORIZON = 2

In [6]:
#We split a test set for testing
# train_test_split(dataX, dataY, test_size=TEST_SIZE)
if SPLIT_INTO_TWO_DAYS:
    LAST_TWO_DAYS_INDEX = -1440*2
    X_train, X_test, y_train, y_test = dataX_nuevo[:LAST_TWO_DAYS_INDEX,:],dataX_nuevo[LAST_TWO_DAYS_INDEX::JUMP,:],\
        dataY[:LAST_TWO_DAYS_INDEX:,0:PREDICTED_HORIZON],dataY[LAST_TWO_DAYS_INDEX::JUMP,0:PREDICTED_HORIZON]
else:
    X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=TEST_SIZE,shuffle=True,
                                                        random_state=42)

    y_test = y_test[:,0:PREDICTED_HORIZON]
    y_train = y_train[:,0:PREDICTED_HORIZON]

print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)
VALIDATION_INDEX = int(len(X_train)*VALIDATION_SIZE)
print("Examples for validation: \n", VALIDATION_INDEX)

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
tf.random.set_seed(42)
# Setup dataset hyperparameters
HORIZON = y_test.shape[1]
WINDOW_SIZE = X_test.shape[1]

# Let's build an LSTM model with the Functional API
inputs = layers.Input(shape=(WINDOW_SIZE))
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activation="relu", return_sequences=True)(x) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128, activation="relu")(x) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_LSTM = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

model_LSTM.summary()

In [ ]:
# Compile model
model_LSTM.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
             metrics=["mae"])

In [ ]:
# Seems when saving the model several warnings are appearing: https://github.com/tensorflow/tensorflow/issues/47554
model_LSTM.fit(X_train[:-VALIDATION_INDEX],
            y_train[:-VALIDATION_INDEX],
            epochs=10,
            verbose=1,
            batch_size=128,
            validation_data=(X_train[-VALIDATION_INDEX:], y_train[-VALIDATION_INDEX:]))

In [ ]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data,verbose=2)
  return tf.squeeze(forecast) # return 1D array of predictions

predictions = make_preds(model_LSTM, X_test)

In [ ]:
if SAVE_RESULTS:
    file_path = 'Resources/Resultados/Agregado/'
    if LOW_DATA:
        file_path += "LowData/"
    documents = ['minuteY','hourY','dayY']
    file = file_path+documents[COMPUTED_OPTION]+"-predictions"+".pkl.gz"
    pickle.dump(predictions, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-test"+".pkl.gz"
    pickle.dump(y_test, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-X"+".pkl.gz"
    pickle.dump(X_test, gzip.open(file, 'wb'))

In [ ]:
print("Resultados comparando directamente las salidas")
if y_test.shape != predictions.shape:
    predictions = np.array(predictions).reshape(y_test.shape)
print(y_test.shape)
print(predictions.shape)
print('\033[1m' + "MSE: " + str(mean_squared_error(y_test,predictions)) + '\033[0m')
print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test,predictions)) + '\033[0m')

In [ ]:
if not MULTI_STEP_FORECAST:
    if SPLIT_INTO_TWO_DAYS:
        index = 0
        period = X_test[::120,:]
        period_results = make_preds(model_LSTM,period)
        period_results_to_plot = np.array(period_results).reshape(48)
        y_test_to_plot = y_test[::120,:].reshape(48)
        plot_predictions_vs_real(predictions=period_results_to_plot,reals=y_test_to_plot)
        print('\033[1m' + "Predicted: " + str(np.sum(period_results_to_plot)) + '\033[0m')
        print('\033[1m' + "Reals: " + str(np.sum(y_test_to_plot)) + '\033[0m')
    else:
        print("Resultados poblacionales cada dos horas")
        print('\033[1m' + "MSE: " + str(mean_squared_error(np.sum(y_test,axis=1),np.sum(predictions,axis=1))) + '\033[0m')
        print('\033[1m' + "MAE: " + str(mean_absolute_error(np.sum(y_test,axis=1),np.sum(predictions,axis=1))) + '\033[0m')
        print("Ejemplos aleatorios")
        # Crear un array de índices
        indices_totales = np.arange(y_test.shape[0])
        # Seleccionar 5 índices aleatorios
        indices_aleatorios = np.random.choice(indices_totales, size=5, replace=False)
        for i in indices_aleatorios:
            if COMPUTED_OPTION == 1:
                END = 24
                STARTED_MINUTE = 0
                previous = np.ones(shape=(24))
                for j in range(0,24):
                    previous[j] = np.sum(X_test[i,:][60*j:60*(j+1)])
            else:
                END = 1440
                STARTED_MINUTE = 1000
                previous = X_test[i,:]

            predictions_to_plot = np.ones(shape=(END + PREDICTED_HORIZON))
            predictions_to_plot[0:END] = previous[:]
            predictions_to_plot[END:] = predictions[i,:]
            y_test_to_plot = np.ones(shape=(END + PREDICTED_HORIZON))
            y_test_to_plot[0:END] = previous[:]
            y_test_to_plot[END:] = y_test[i,:]
            plot_predictions_vs_real(predictions_to_plot,y_test_to_plot)



In [ ]:
if MULTI_STEP_FORECAST:
    day_before = np.array(X_test[0:1,:],dtype=np.float32)
    predicted_values_day_one = np.ones(shape=(WINDOW_SIZE,1),dtype=np.float32)
    for i in range(0,WINDOW_SIZE):
        prediction = float(make_preds(model_LSTM, day_before))
        day_before[:,0:-1] = day_before[:,1:]
        day_before[:,-1] = prediction
        predicted_values_day_one[i,0] = prediction

    SECOND_DAY = X_test.shape[1]
    day_before = np.array(X_test[SECOND_DAY:SECOND_DAY+1,:],dtype=np.float32)
    predicted_values_day_two = np.ones(shape=(WINDOW_SIZE,1),dtype=np.float32)
    for i in range(0,WINDOW_SIZE):
        prediction = float(make_preds(model_LSTM, day_before))
        day_before[:,0:-1] = day_before[:,1:]
        day_before[:,-1] = prediction
        predicted_values_day_two[i,0] = prediction

In [ ]:
if MULTI_STEP_FORECAST:
    print("Predecidos : " + str(int(np.sum(predicted_values_day_one[:,0]))))
    print("Reales : " + str(int(np.sum(y_test[0:WINDOW_SIZE,0]))))
    if COMPUTED_OPTION < 2:
        plot_predictions_vs_real(predicted_values_day_one[:,0],y_test[0:WINDOW_SIZE,0])

In [ ]:
if MULTI_STEP_FORECAST:
    print("Predecidos : " + str(int(np.sum(predicted_values_day_two[:,0]))))
    print("Reales : " + str(int(np.sum(y_test[SECOND_DAY:,0]))))
    if COMPUTED_OPTION < 2:
        plot_predictions_vs_real(predicted_values_day_two[:,0],y_test[SECOND_DAY:,0])